# Get stock data from API

In [1]:
import requests
import pandas as pd
import bokeh.plotting as bk
from bokeh.models import HoverTool, BoxZoomTool, PanTool, ZoomInTool, ZoomOutTool, ResetTool, Label
import numpy as np

API_URL = 'https://api.iextrading.com/1.0/stock/{}/chart/5y'

In [2]:
# Make the API request and load json data into DataFrame
symbol = 'amzn'
res = requests.get(API_URL.format(symbol))
data = res.json()
df = pd.DataFrame(data)

df.sample(10)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
160,0.99,-0.206067,0.311,319.32,2014-08-12,321.3100,"Aug 12, 14",316.95,318.89,1916143,1916143,319.1125
1078,41.18,2.609523,2.919,1451.75,2018-04-05,1459.5600,"Apr 5, 18",1427.07,1441.99,6413459,6413459,1445.1900
246,-0.04,-0.235903,-0.013,307.32,2014-12-12,310.6399,"Dec 12, 14",303.01,303.99,3162322,3162322,308.1892
356,-4.00,0.063227,-0.927,427.63,2015-05-22,432.4400,"May 22, 15",427.61,431.55,2020629,2020629,429.5928
1013,-16.63,1.907683,-1.402,1169.47,2017-12-29,1184.0000,"Dec 29, 17",1167.50,1182.35,2688391,2688391,1174.7000
542,-9.10,0.305321,-1.704,525.00,2016-02-18,541.2000,"Feb 18, 16",523.73,541.19,4735008,4735008,532.5231
195,-4.98,-0.210691,-1.544,317.46,2014-10-01,322.1650,"Oct 1, 14",315.55,322.04,3096735,3096735,317.8827
861,8.81,1.437469,0.907,980.35,2017-05-24,981.0000,"May 24, 17",970.23,976.00,2463052,2463052,976.2573
873,-13.40,1.399080,-1.370,964.91,2017-06-12,975.9500,"Jun 12, 17",945.00,967.00,9447233,9447233,960.5778
565,6.50,0.393536,1.173,560.48,2016-03-22,562.7600,"Mar 22, 16",545.05,545.11,4000564,4000564,557.7667


In [3]:
#Add a new column in the shape of the original data
seqs = np.arange(df.shape[0])
df['seqs'] = pd.Series(seqs)

In [4]:
# Add a mid column that is midpoint between open and close
df['mid'] = df.apply(lambda x: (x['open'] + x['close']) / 2, axis=1)

In [5]:
# Add height column after checking if close is greater than open
df['height'] = df.apply(lambda x: x['close'] - x['open'] if x['close'] != x['open'] else 0.01, axis=1)

In [6]:
df.head(2)

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap,seqs,mid,height
0,7.01,0.00000,1.774,402.20,2013-12-20,404.72,"Dec 20, 13",395.78,396.55,5038537,5038537,401.1329,0,399.375,5.65
1,0.72,0.00179,0.179,402.92,2013-12-23,405.00,"Dec 23, 13",399.20,403.69,2661823,2661823,402.2857,1,403.305,-0.77


In [7]:
# some stuff to format plot
inc = df.close > df.open #Series where stocks closed higher than they opened
dec = df.close < df.open #Series in which stocks closed lower than they opened
w = .3     #line width for plot

In [8]:
#Define sources for increasing and decreasing stocks
sourceInc = bk.ColumnDataSource(df.loc[inc])
sourceDec = bk.ColumnDataSource(df.loc[dec])

In [9]:
#Define the Bokeh tools to include in the figure
hover = HoverTool(
    tooltips=[
        ('Date', '@date'),
        ('Low', '@low'),
        ('High', '@high'),
        ('Open', '@open'),
        ('Close', '@close'),
        ('Percent', '@changePercent')
    ]
)

In [10]:
TOOLS = [hover, BoxZoomTool(), PanTool(), ZoomInTool(), ZoomOutTool(), ResetTool()]

In [11]:
#Define size and layout of figure
p = bk.figure(plot_width=1200, plot_height=800, title='Amazon', tools=TOOLS, toolbar_location='above')
p.xaxis.major_label_orientation = np.pi/4
p.grid.grid_line_alpha = w
descriptor = Label(x=70,  y=70, text='Amazon Stock Price Over Time')
p.add_layout(descriptor)

In [12]:
#Create segments for price increases
p.segment(df.seqs[inc], df.high[inc], df.seqs[inc], df.low[inc], color='green')
#Create segments for price decreases
p.segment(df.seqs[dec], df.high[dec], df.seqs[dec], df.low[dec], color='red')

GlyphRenderer(id='1048', ...)

In [13]:
#Create rects for price increases
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='green', line_color='green', source=sourceInc)
#Create rects for price decreases
p.rect(x='seqs', y='mid', width=w, height='height', fill_color='red', line_color='red', source=sourceDec)

GlyphRenderer(id='1060', ...)

In [14]:
# bk.show(p)
bk.save(p, './stock_plot.html')

/home/scott/.local/share/virtualenvs/stock_portfolio-tsVmF7JH/lib/python3.6/site-packages/bokeh/io/saving.py:127: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/home/scott/.local/share/virtualenvs/stock_portfolio-tsVmF7JH/lib/python3.6/site-packages/bokeh/io/saving.py:140: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'/home/scott/codefellows/401/projects/stock_portfolio/stock_plot.html'

In [19]:
bk.show(p)